## Описание и проверка гипотез

### Импорты

In [9]:
import os
import kagglehub
import cv2
import src.image_classification.features as feat

from scipy.stats import mannwhitneyu, spearmanr, kruskal, levene, ttest_ind, pearsonr, chi2_contingency, f_oneway

### Основной путь и классы, с которыми будет идти работа

In [10]:
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")
classes = os.listdir(os.path.join(path, "garbage-dataset"))

### Гипотеза №1 О наличие картона на картинке можно судить по скосу желтого цвета.

В данной гипотезе мы предполагаем, что цветовая характеристика изображения, а именно наличие желтого цвета, может служить индикатором присутствия картона на картинке. Исходя из наблюдений, мы предполагаем, что изображения, содержащие картон, будут иметь более выраженный желтый оттенок по сравнению с изображениями, не содержащими этот материал.


- Способы проверки:

    - **t-критерий Стьюдента (t-test)**: Этот метод будет использован для сравнения средней доли желтого цвета между двумя группами. Мы проверим нулевую гипотезу о том, что средние значения доли желтого цвета в обеих группах не отличаются.

    - **ANOVA (дисперсионный анализ)**: Если у нас есть более двух групп (например, различные виды картона и другие материалы), ANOVA поможет определить, есть ли значимые различия в количестве желтого цвета между всеми группами.

    - **Критерий Манна-Уитни (U-тест)**: Непараметрический тест для сравнения распределений двух групп (в данном случае картон и другие классы).

In [11]:
cardboard_path = os.path.join(path + "/garbage-dataset", "cardboard")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "cardboard"]

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = cv2.imread(filepath)
        if image is not None: 
            images.append(image)
    return images

yellow_feature = feat.AmountOfYellow()

cardboard_images = load_images_from_folder(cardboard_path)
cardboard_yellow_amounts = [yellow_feature(img) for img in cardboard_images]

other_yellow_amounts = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_yellow_amounts.extend([yellow_feature(img) for img in other_images])

# Проверка гипотез
# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(cardboard_yellow_amounts, other_yellow_amounts, equal_var=False)

# 2. ANOVA
anova_stat, anova_pvalue = f_oneway(cardboard_yellow_amounts, other_yellow_amounts)

# 3. Критерий Манна-Уитни
mann_stat, mann_pvalue = mannwhitneyu(cardboard_yellow_amounts, other_yellow_amounts, alternative='two-sided')

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "ANOVA": {"statistic": anova_stat, "p-value": anova_pvalue},
    "Mann-Whitney": {"statistic": mann_stat, "p-value": mann_pvalue},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Итог: гипотеза подтверждена: наличие картона связано c желтым цветом.")
else:
    print("Итог: гипотеза не подтверждена: наличие картона не связано c желтым цветом.")

t-test: Statistic = 3.683066343249196, p-value = 0.00023621963453461276
ANOVA: Statistic = 16.865194578791744, p-value = 4.0292196417557166e-05
Mann-Whitney: Statistic = 22080315.5, p-value = 1.0126318223771204e-146
Итог: гипотеза подтверждена: наличие картона связано c желтым цветом.


### Гипотеза №2 О наличие металла на картинке можно судить по скосу серого (серебряного) цвета.

В данной гипотезе мы предполагаем, что цветовая характеристика изображения, а именно наличие серого или серебряного цвета, может служить индикатором присутствия металла на картинке. Мы предполагаем, что изображения, содержащие металл, будут иметь более выраженный серый оттенок по сравнению с изображениями других классов.

- Способы проверки:

    - **t-критерий Стьюдента (t-test)**: Этот метод будет использован для сравнения средней доли серебряного цвета между двумя группами. Мы проверим нулевую гипотезу о том, что средние значения доли серебряного цвета в обеих группах не отличаются.

    - **ANOVA (дисперсионный анализ)**: Если у нас есть более двух групп (например, различные виды металлов и другие материалы), ANOVA поможет определить, есть ли значимые различия в количестве серебряного цвета между всеми группами. 

    - **Критерий Манна-Уитни (U-тест)**: Непараметрический тест для сравнения распределений двух групп (в данном случае металл и другие классы).

In [12]:
metal_path = os.path.join(path + "/garbage-dataset", "metal")
other_paths = [os.path.join(path + "/garbage-dataset", cls) for cls in classes if cls != "metal"]

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = cv2.imread(filepath)
        if image is not None:
            images.append(image)
    return images

silver_feature = feat.AmountOfSilver()

metal_images = load_images_from_folder(metal_path)
metal_silver_amounts = [silver_feature(img) for img in metal_images]

other_silver_amounts = []
for other_path in other_paths:
    other_images = load_images_from_folder(other_path)
    other_silver_amounts.extend([silver_feature(img) for img in other_images])

# Проверка гипотез
# 1. t-критерий Стьюдента
t_stat, t_pvalue = ttest_ind(metal_silver_amounts, other_silver_amounts, equal_var=False)

# 2. ANOVA
anova_stat, anova_pvalue = f_oneway(metal_silver_amounts, other_silver_amounts)

# 3. Критерий Манна-Уитни
mann_stat, mann_pvalue = mannwhitneyu(metal_silver_amounts, other_silver_amounts, alternative='two-sided')

results = {
    "t-test": {"statistic": t_stat, "p-value": t_pvalue},
    "ANOVA": {"statistic": anova_stat, "p-value": anova_pvalue},
    "Mann-Whitney": {"statistic": mann_stat, "p-value": mann_pvalue},
}

for test, result in results.items():
    print(f"{test}: Statistic = {result['statistic']}, p-value = {result['p-value']}")

alpha = 0.05
hypothesis_results = {test: result['p-value'] < alpha for test, result in results.items()}

if all(hypothesis_results.values()):
    print("Гипотеза подтверждена: наличие металла связано c серебряным цветом.")
else:
    print("Гипотеза не подтверждена: наличие металла не связано c серебряным цветом.")

t-test: Statistic = -2.284424004913432, p-value = 0.022401779559765453
ANOVA: Statistic = 0.5733189930562239, p-value = 0.448952071385762
Mann-Whitney: Statistic = 10996543.5, p-value = 5.262970510833023e-16
Гипотеза не подтверждена: наличие металла не связано c серебряным цветом.
